In [1]:
#######################################################################################################
# Summary
# 1. PyTorch Multi-GPU example
# 2. On-the-fly data-augmentation (random crop, random flip)
#######################################################################################################

In [2]:
MULTI_GPU = False

In [3]:
import os
import sys
import time
import multiprocessing
import numpy as np
import pandas as pd
import torch
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics.ranking import roc_auc_score
from sklearn.model_selection import train_test_split
from PIL import Image
from common.utils import download_data_chextxray, get_imgloc_labels, get_train_valid_test_split
from common.utils import compute_roc_auc, get_cuda_version, get_cudnn_version, get_gpu_name
from common.utils import yield_mb
from common.params_dense import *

In [4]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("PyTorch: ", torch.__version__)
print("Numpy: ", np.__version__)
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.5.4 |Anaconda custom (64-bit)| (default, Nov 20 2017, 18:44:38) 
[GCC 7.2.0]
PyTorch:  0.4.0
Numpy:  1.14.1
GPU:  ['Tesla V100-PCIE-16GB', 'Tesla V100-PCIE-16GB', 'Tesla V100-PCIE-16GB', 'Tesla V100-PCIE-16GB']
CUDA Version 9.0.176
CuDNN Version  7.0.5


In [5]:
CPU_COUNT = multiprocessing.cpu_count()
GPU_COUNT = len(get_gpu_name())
print("CPUs: ", CPU_COUNT)
print("GPUs: ", GPU_COUNT)

CPUs:  24
GPUs:  4


In [6]:
# Model-params
IMAGENET_RGB_MEAN_TORCH = [0.485, 0.456, 0.406]
IMAGENET_RGB_SD_TORCH = [0.229, 0.224, 0.225]
# Paths
CSV_DEST = "chestxray"
IMAGE_FOLDER = os.path.join(CSV_DEST, "images")
LABEL_FILE = os.path.join(CSV_DEST, "Data_Entry_2017.csv")
print(IMAGE_FOLDER, LABEL_FILE)

chestxray/images chestxray/Data_Entry_2017.csv


In [7]:
# Manually scale to multi-gpu
assert torch.cuda.is_available()
_DEVICE = torch.device("cuda:0")
# enables cudnn's auto-tuner
torch.backends.cudnn.benchmark=True
if MULTI_GPU:
    LR *= GPU_COUNT 
    BATCHSIZE *= GPU_COUNT

In [8]:
%%time
# Download data
# Wall time: 17min 58s
print("Please make sure to download")
print("https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy")
download_data_chextxray(CSV_DEST)

Please make sure to download
https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy
Data already exists
CPU times: user 606 ms, sys: 239 ms, total: 845 ms
Wall time: 845 ms


In [9]:
#####################################################################################################
## Data Loading

In [10]:
# Normalise by imagenet mean/sd
normalize = transforms.Normalize(IMAGENET_RGB_MEAN_TORCH,
                                 IMAGENET_RGB_SD_TORCH)

In [11]:
class XrayData(Dataset):
    def __init__(self, img_dir, lbl_file, patient_ids, transform=None):
        
        self.img_locs, self.labels = get_imgloc_labels(img_dir, lbl_file, patient_ids)
        self.transform = transform
        print("Loaded {} labels and {} images".format(len(self.labels), len(self.img_locs)))
    
    def __getitem__(self, idx):
        im_file = self.img_locs[idx]
        im_rgb = Image.open(im_file)
        label = self.labels[idx]
        if self.transform is not None:
            im_rgb = self.transform(im_rgb)
        return im_rgb, torch.FloatTensor(label)
        
    def __len__(self):
        return len(self.img_locs)

In [12]:
def no_augmentation_dataset(img_dir, lbl_file, patient_ids, normalize):
    dataset = XrayData(img_dir, lbl_file, patient_ids,
                       transform=transforms.Compose([
                           transforms.Resize(WIDTH),
                           transforms.ToTensor(),  
                           normalize]))
    return dataset

In [13]:
train_set, valid_set, test_set = get_train_valid_test_split(TOT_PATIENT_NUMBER)

train:21563 valid:3080 test:6162


In [14]:
# Dataset for training
train_dataset = XrayData(img_dir=IMAGE_FOLDER,
                         lbl_file=LABEL_FILE,
                         patient_ids=train_set,
                         transform=transforms.Compose([
                             transforms.RandomResizedCrop(size=WIDTH),
                             transforms.RandomHorizontalFlip(),
                             transforms.ToTensor(),  # need to convert image to tensor!
                             normalize]))

Loaded 87306 labels and 87306 images


In [15]:
valid_dataset = no_augmentation_dataset(IMAGE_FOLDER, LABEL_FILE, valid_set, normalize)
test_dataset = no_augmentation_dataset(IMAGE_FOLDER, LABEL_FILE, test_set, normalize)

Loaded 7616 labels and 7616 images
Loaded 17198 labels and 17198 images


In [16]:
#####################################################################################################
## Helper Functions

In [17]:
def get_symbol(out_features=CLASSES, multi_gpu=MULTI_GPU):
    model = models.densenet.densenet121(pretrained=True)
    # Replace classifier (FC-1000) with (FC-14)
    model.classifier = nn.Sequential(
        nn.Linear(model.classifier.in_features, out_features), 
        nn.Sigmoid())
    if multi_gpu:
        model = nn.DataParallel(model)
    # CUDA
    model.to(_DEVICE)  
    return model

In [18]:
def init_symbol(sym, lr=LR):
    # BCE Loss since classes not mutually exclusive + Sigmoid FC-layer
    cri = nn.BCELoss().cuda()
    opt = optim.Adam(sym.parameters(), lr=lr, betas=(0.9, 0.999))
    sch = ReduceLROnPlateau(opt, factor=0.1, patience=5, mode='min')
    return opt, cri, sch 

In [19]:
def train_epoch(model, dataloader, optimizer, criterion, epoch):
    model.train()
    print("Training epoch {}".format(epoch+1))
    loss_val = 0
    loss_cnt = 0
    for data, target in dataloader:
        # Get samples
        data, target = data.to(_DEVICE), target.cuda(non_blocking=True)
        # Forwards
        output = model(data)
        # Loss
        loss = criterion(output, target)
        # Back-prop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()   
         # Log the loss
        loss_val += loss.item()
        loss_cnt += 1
    print("Training loss: {0:.4f}".format(loss_val/loss_cnt))

In [20]:
def valid_epoch(model, dataloader, criterion, epoch, phase='valid', cl=CLASSES):
    model.eval()
    if phase == 'testing':
        print("Testing epoch {}".format(epoch+1))
    else:
        print("Validating epoch {}".format(epoch+1))
    # Don't save gradients
    with torch.no_grad():
        out_pred = torch.FloatTensor().cuda()
        out_gt = torch.FloatTensor().cuda()
        loss_val = 0
        loss_cnt = 0
        for data, target in dataloader:
            # Get samples
            data, target = data.to(_DEVICE), target.cuda(non_blocking=True)
             # Forwards
            output = model(data)
            # Loss
            loss = criterion(output, target)
            # Log the loss
            loss_val += loss.item()
            loss_cnt += 1
            # Log for AUC
            out_pred = torch.cat((out_pred, output.data), 0)
            out_gt = torch.cat((out_gt, target.data), 0)

        loss_mean = loss_val/loss_cnt
        out_gt = out_gt.cpu().numpy()
        out_pred = out_pred.cpu().numpy()    
    
    if phase == 'testing':
        print("Test-Dataset loss: {0:.4f}".format(loss_mean))
        print("Test-Dataset AUC: {0:.4f}".format(compute_roc_auc(out_gt, out_pred, cl)))
    else:
        print("Validation loss: {0:.4f}".format(loss_mean))
        print("Validation AUC: {0:.4f}".format(compute_roc_auc(out_gt, out_pred, cl)))
    return loss_mean

In [21]:
# DataLoaders
# pin_memory brings errors on 4 GPUs
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCHSIZE,
                          shuffle=True, num_workers=CPU_COUNT, pin_memory=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=16*BATCHSIZE,
                          shuffle=False, num_workers=CPU_COUNT)
test_loader = DataLoader(dataset=test_dataset, batch_size=16*BATCHSIZE,
                         shuffle=False, num_workers=CPU_COUNT)

In [22]:
#####################################################################################################
## Train CheXNet

In [23]:
%%time
# Load symbol
chexnet_sym = get_symbol()

/anaconda/envs/py35/lib/python3.5/site-packages/torchvision-0.2.1-py3.5.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


CPU times: user 4.45 s, sys: 1.51 s, total: 5.96 s
Wall time: 8.12 s


In [24]:
%%time
# Load optimiser, loss
optimizer, criterion, scheduler = init_symbol(chexnet_sym)

CPU times: user 1.42 ms, sys: 412 µs, total: 1.83 ms
Wall time: 1.84 ms


In [25]:
%%time
# 1 GPU - Main training loop: 27min9s
# 4 GPU - Main training loop: 12min27s
loss_min = float("inf")    
# Main train/val loop
for j in range(EPOCHS):
    stime = time.time()
    train_epoch(chexnet_sym, train_loader, optimizer, criterion, j)
    loss_val = valid_epoch(chexnet_sym, valid_loader, criterion, j)
    # LR Schedule
    scheduler.step(loss_val)
    etime = time.time()
    print("Epoch time: {0:.0f} seconds".format(etime-stime))
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

Training epoch 1
Training loss: 0.1744
Validating epoch 1
Validation loss: 0.1393
Full AUC [0.7844880151387721, 0.8686341336343506, 0.790397107078283, 0.9159155212421788, 0.8996286183103711, 0.8746895630971845, 0.779259062541495, 0.7283131738101498, 0.6484075352765513, 0.784317365927381, 0.7149508102394756, 0.7817639842970813, 0.7961877482530276, 0.8557432514134676]
Validation AUC: 0.8016
Epoch time: 344 seconds
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Training epoch 2
Training loss: 0.1569
Validating epoch 2
Validation loss: 0.1384
Full AUC [0.796584796397533, 0.8883782688606202, 0.7798704584179706, 0.9172255791433735, 0.8988566279047742, 0.9006488767699854, 0.7623438805876885, 0.7952800420720484, 0.654006902722345, 0.8006906278659375, 0.7260984045380916, 0.7827990559494197, 0.8082725519875251, 0.8718641754908036]
Validation AUC: 0.8131
Epoch time: 323 seconds
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Training epoch 3
Training loss: 0.1542
Validating epoch 3
Validation lo

In [26]:
#####################################################################################################
## Test CheXNet

In [27]:
%%time
# Load model for testing
# I comment this out to create a fair test against Keras
#chexnet_sym_test = get_symbol()
#chkpt = torch.load("best_chexnet.pth.tar")
#chexnet_sym_test.load_state_dict(chkpt['state_dict'])

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 11.2 µs


In [28]:
%%time
# AUC: 0.8211 Single-GPU
# AUC: 0.8078 Multi-GPU
#test_loss = valid_epoch(chexnet_sym_test, test_loader, criterion, -1, 'testing')
test_loss = valid_epoch(chexnet_sym, test_loader, criterion, -1, 'testing')

Testing epoch 0
Test-Dataset loss: 0.1532
Full AUC [0.8148510498599838, 0.8597881471794567, 0.799994136552366, 0.9067576711963495, 0.8871971077304283, 0.9170567258058132, 0.7362130257111875, 0.8788233080685911, 0.625676054810796, 0.841234634151767, 0.7583817816727334, 0.8125815933985783, 0.7749702928189925, 0.8814175200671013]
Test-Dataset AUC: 0.8211
CPU times: user 11.4 s, sys: 8.04 s, total: 19.5 s
Wall time: 33.4 s


In [29]:
#####################################################################################################
## Synthetic Data (Pure Training)

In [30]:
# Test on fake-data -> no IO lag
batch_in_epoch = len(train_dataset.labels)//BATCHSIZE
tot_num = batch_in_epoch * BATCHSIZE
print(tot_num)

87296


In [31]:
fake_X = torch.tensor(np.random.rand(tot_num, 3, 224, 224).astype(np.float32))
fake_y = torch.tensor(np.random.rand(tot_num, CLASSES).astype(np.float32))

In [32]:
%%time
# 1 GPU - Synthetic data: 25min23s
# 4 GPU - Synthetic data: 8min23s
for j in range(EPOCHS):
    train_epoch(chexnet_sym, 
                yield_mb(fake_X, fake_y, BATCHSIZE, shuffle=False),
                optimizer, 
                criterion,
                j)

Training epoch 1
Training loss: 0.7024
Training epoch 2
Training loss: 0.6934
Training epoch 3
Training loss: 0.6934
Training epoch 4
Training loss: 0.6934
Training epoch 5
Training loss: 0.6933
CPU times: user 22min 33s, sys: 2min 47s, total: 25min 21s
Wall time: 25min 23s
